[View in Colaboratory](https://colab.research.google.com/github/indes/hesay_source/blob/master/static/ipynb/python_decorate.ipynb)

# ⚗️Python 装饰器学习

In [0]:
def deco(func):
  '''
  该函数参数为另外一个函数
  '''
	def inner():
		print('running %s' % func.__name__)
		return func()
	return inner

In [0]:
def func():
  print('123')

In [4]:
tele = deco(func)
tele()

running func
123
